In [1]:
import pomegranate
import numpy as np
import pandas as pd
import random

# Load data

In [2]:
df = pd.read_csv("../../Data/Data/Strata_Train_NPS_cases/Train_NPS_evlog.csv")
print(len(df))
df.head()

5456


,case_id,case_type,task_id,task_tasksubtype,ressource,ressource_role,case_owner_role,case_topic,case_requires_rework__c,case_closure_code__c,...,task_count,task_number,date_case_created,date_case_closed,date_task_created,year,month,day,weekday,hour
0,1485898,Service Request,1,Email,y_7,b_1,b_1,d_2,False,Changed subscription,...,2,1,2018-02-12 11:40:54,2018-02-18 23:01:09+00:00,2018-02-12 11:42:22+00:00,2018,2,12,0,11
1,1485898,Service Request,2,Email,y_7,b_1,b_1,d_2,False,Changed subscription,...,2,2,2018-02-12 11:40:54,2018-02-18 23:01:09+00:00,2018-02-12 11:57:20+00:00,2018,2,12,0,11
2,1491822,Service Request,5,Email,y_9,i_1,i_1,d_2,False,Information/guidance given,...,3,1,2018-02-20 10:17:59,2018-03-05 23:00:51+00:00,2018-02-20 10:20:20+00:00,2018,2,20,1,10
3,1491822,Service Request,10,Email,y_9,i_1,i_1,d_2,False,Information/guidance given,...,3,2,2018-02-20 10:17:59,2018-03-05 23:00:51+00:00,2018-02-21 11:19:09+00:00,2018,2,21,2,11
4,1491822,Service Request,19,Email,y_9,i_1,i_1,d_2,False,Information/guidance given,...,3,3,2018-02-20 10:17:59,2018-03-05 23:00:51+00:00,2018-02-23 10:30:44+00:00,2018,2,23,4,10


# Simplify

In [3]:
df = df[["case_id","case_topic","task_number","task_tasksubtype"]]
df.head()

,case_id,case_topic,task_number,task_tasksubtype
0,1485898,d_2,1,Email
1,1485898,d_2,2,Email
2,1491822,d_2,1,Email
3,1491822,d_2,2,Email
4,1491822,d_2,3,Email


## Transform into a one-dimensional array with variable-length traces. 

### Add absorption state "END" to the trace (multiple times, to avoid HOMC problems)

In [4]:
data = [] #np.array([])
#prepare for MC
for caseid in list(set(df.case_id.values)):
    
    trace = df.loc[df.case_id == caseid]["task_tasksubtype"].values
    trace = np.append(trace,"END")
    trace = np.append(trace,"END")
    trace = np.append(trace,"END")
    trace = np.append(trace,"END")
    trace = np.append(trace,"END")
    
    #data = np.concatenate((data, trace),axis=0)
    #data = np.append([data],[[trace]])
    data.append(trace)
    
#convert from list to array
traindata = np.asarray(data)
traindata

C:\Users\Mike\anaconda3\envs\TF2_CUD11_CRM_PAPER\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


array([array(['Interaction', 'Email', 'Email', 'END', 'END', 'END', 'END', 'END'],
      dtype=object),
       array(['Interaction', 'Email', 'END', 'END', 'END', 'END', 'END'],
      dtype=object),
       array(['Interaction', 'Email', 'END', 'END', 'END', 'END', 'END'],
      dtype=object),
       ...,
       array(['Email', 'Email', 'Interaction', 'Email', 'Email', 'END', 'END',
       'END', 'END', 'END'], dtype=object),
       array(['Interaction', 'Email', 'END', 'END', 'END', 'END', 'END'],
      dtype=object),
       array(['Interaction', 'Email', 'END', 'END', 'END', 'END', 'END'],
      dtype=object)], dtype=object)

In [5]:
np.shape(traindata)

(1898,)

# Fit a markov chain

In [6]:
"""
Generate markov chain with pomegranate
"""

#HOMC
#max_trace_len = 5
#model = pomegranate.MarkovChain.from_samples(X=traindata, 
#                                  weights=None, 
#                                  k=max_trace_len)

#MC
model = pomegranate.MarkovChain.from_samples(X=traindata)#, k=1)


In [7]:
"""
Inspect
"""
#unconditional probabilities
model.distributions[0]

#all probabilities#
#model.distributions

{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Task-Reminder" : 0.0,
            "END" : 0.0,
            "Interaction" : 0.9188619599578504,
            "Email" : 0.08113804004214963
        }
    ],
    "frozen" : false
}

In [8]:
cond_prob = model.distributions[1]

cond_prob.to_dict()["table"]

[['Task-Reminder', 'Task-Reminder', '0.0909090909090909'],
 ['Task-Reminder', 'END', '0.22727272727272727'],
 ['Task-Reminder', 'Interaction', '0.04545454545454546'],
 ['Task-Reminder', 'Email', '0.6363636363636364'],
 ['END', 'Task-Reminder', '0.0'],
 ['END', 'END', '1.0'],
 ['END', 'Interaction', '0.0'],
 ['END', 'Email', '0.0'],
 ['Interaction', 'Task-Reminder', '0.001078167115902965'],
 ['Interaction', 'END', '0.02048517520215633'],
 ['Interaction', 'Interaction', '0.017250673854447448'],
 ['Interaction', 'Email', '0.9611859838274932'],
 ['Email', 'Task-Reminder', '0.005029337803855825'],
 ['Email', 'END', '0.5183012014529198'],
 ['Email', 'Interaction', '0.02179379715004191'],
 ['Email', 'Email', '0.45487566359318243']]

In [9]:

transition_table = pd.DataFrame(model.distributions[1].to_dict()["table"], columns=["from","to","prob"])
transition_table["prob"] = pd.to_numeric(transition_table["prob"], downcast="float")

adj_matrix = transition_table.pivot(index='from', columns='to', values="prob").round(2)
adj_matrix

to,END,Email,Interaction,Task-Reminder
from,,,,
END,1.00,0.00,0.00,0.00
Email,0.52,0.45,0.02,0.01
Interaction,0.02,0.96,0.02,0.00
Task-Reminder,0.23,0.64,0.05,0.09


# Sample from the distribution

In [10]:
"""
First-order markov chain
"""

model = pomegranate.MarkovChain.from_samples(X=traindata)

predictions=[]
num_predictions = 4

for i in range(0,num_predictions):
    
    # sample untill length k from the markov chain
    sample=model.sample(5)
    print(sample)
    predictions.append(sample)
    
predictions = np.asarray(predictions)


['Email', 'END', 'END', 'END', 'END']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'END', 'END', 'END']


In [11]:
"""
Higher-order markov chain
"""

k = 5
num_predictions = 200

model = pomegranate.MarkovChain.from_samples(X=traindata, 
                                  weights=None, 
                                  k=k)
predictions=[]

for i in range(0,num_predictions):
    
    # sample untill length k from the markov chain
    sample=model.sample(k)
    print(sample)
    predictions.append(sample)
    
predictions = np.asarray(predictions)

['Interaction', 'Email', 'Interaction', 'Interaction', 'Interaction']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'Email', 'END', 'END']
['Email', 'Email', 'Email', 'Email', 'Email']
['Interaction', 'Email', 'Email', 'END', 'END']
['Interaction', 'Email', 'Email', 'END', 'END']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'Interaction', 'Interaction', 'Email']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'Email', 'END', 'END']
['Interaction', 'Email', 'Email', 'Interaction', 'Email']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'Email', 'END', 'END']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'Email', 'END', 'END']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'END', 'END', 'END']
['Interaction', 'Email', 'Email', 'Email', 'END']
['Interact

# Evaluation

In [12]:
"""
Procedure:
- Use transition matrices
- Train and holdout
- Compare fit in train to empirical holdout
    - flatten / vectorize matrices
    - calculate KL divergence
"""

'\nProcedure:\n- Use transition matrices\n- Train and holdout\n- Compare fit in train to empirical holdout\n    - flatten / vectorize matrices\n    - calculate KL divergence\n'

In [13]:
def DL_distance():
    DL = _
    return DL

def euclidean_distance(p,q):
    dist = np.linalg.norm(p-q)
    return dist

def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

In [14]:
#make a holdout sample

from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(traindata, test_size=0.3)#, random_state=42)

In [15]:
k_size = 1

#Train the baseline model
model_test = pomegranate.MarkovChain.from_samples(X=X_test, 
                                  weights=None, 
                                  k=k_size)

#Train the model
model_train = pomegranate.MarkovChain.from_samples(X=X_train, 
                                  weights=None, 
                                  k=k_size)

# 1. Generate the two transition matrices

In [16]:
test_trans = pd.DataFrame(model_test.distributions[1].to_dict()["table"], columns=["from","to","prob"])
test_trans["prob"] = pd.to_numeric(test_trans["prob"], downcast="float")

train_trans = pd.DataFrame(model_train.distributions[1].to_dict()["table"], columns=["from","to","prob"])
train_trans["prob"] = pd.to_numeric(train_trans["prob"], downcast="float")


## 1.1 Store them so they can be used in the paper

In [63]:
adj_matrix_train = train_trans.pivot(index='from', columns='to', values="prob").round(2)
adj_matrix_train = adj_matrix_train.sort_index(axis='columns', level='from', ascending=False)
adj_matrix_train = adj_matrix_train.sort_index(axis='rows', level='from', ascending=False)
adj_matrix_train.to_csv("../tables/activity_sequence_params_train.csv")
adj_matrix_train.to_csv("../tables/Final_models/activity_sequence_parameters_train.csv")
adj_matrix_train

to,Task-Reminder,Interaction,Email,END
from,,,,
Task-Reminder,0.10,0.05,0.60,0.25
Interaction,0.00,0.02,0.96,0.02
Email,0.01,0.02,0.46,0.52
END,0.00,0.00,0.00,1.00


In [64]:
adj_matrix_test = test_trans.pivot(index='from', columns='to', values="prob").round(2)
adj_matrix_test = adj_matrix_test.sort_index(axis='columns', level='from', ascending=False)
adj_matrix_test = adj_matrix_test.sort_index(axis='rows', level='from', ascending=False)
adj_matrix_test.to_csv("../tables/Final_models/activity_sequence_params_test.csv")
adj_matrix_train.to_csv("../tables/Final_models/activity_sequence_parameters_test.csv")
adj_matrix_test

to,Task-Reminder,Interaction,Email,END
from,,,,
Task-Reminder,0.0,0.00,1.00,0.00
Interaction,0.0,0.01,0.97,0.02
Email,0.0,0.02,0.45,0.52
END,0.0,0.00,0.00,1.00


## 1.2 Get the base rates 

In [50]:
#### train

In [56]:
first_ev = []

for trace in X_train:
    #print(trace[0])
    first_ev.append(trace[0])

train_bases = pd.DataFrame({"activity":first_ev})
train_bases.value_counts()

activity   
Interaction    1226
Email           102
dtype: int64

In [57]:
n = 1744 + 154
[np.round(1744/n,decimals=2),np.round(154/n,decimals=2),0,0]

[0.92, 0.08, 0, 0]

In [58]:
#### test

In [59]:
first_ev = []

for trace in X_test:
    #print(trace[0])
    first_ev.append(trace[0])

test_bases = pd.DataFrame({"activity":first_ev})
test_bases.value_counts()

activity   
Interaction    518
Email           52
dtype: int64

In [62]:
n = 518 + 52
[np.round(518/n,decimals=2),np.round(52/n,decimals=2),0,0]

[0.91, 0.09, 0, 0]

# 2. Vectorize them

In [19]:
# sort them so they have the same order
train_trans = train_trans.sort_values(["to", "from"], ascending = (False, False))
train_trans

test_trans = test_trans.sort_values(["to", "from"], ascending = (False, False))
test_trans

,from,to,prob
0,Task-Reminder,Task-Reminder,0.100000
8,Interaction,Task-Reminder,0.001786
12,Email,Task-Reminder,0.007339
4,END,Task-Reminder,0.000000
2,Task-Reminder,Interaction,0.000000
10,Interaction,Interaction,0.016071
14,Email,Interaction,0.021101
6,END,Interaction,0.000000
3,Task-Reminder,Email,0.700000
11,Interaction,Email,0.960714


In [20]:
#normalize to avoid zeros
test_prob_norm = (test_trans["prob"].values+0.0001)/np.sum(test_trans["prob"].values)
train_prob_norm = (train_trans["prob"].values+0.0001)/np.sum(train_trans["prob"].values)

#no normalization
test_prob = test_trans["prob"]
train_prob = train_trans["prob"]


# Calculate distance

In [21]:
#KL divergence: Entropy between two probability densities
kl_divergence(train_prob_norm, 
              test_prob_norm)

0.12426761

In [22]:
#L2 norm between two vectors
euclidean_distance(train_prob.values, test_prob.values)

0.15356018

In [23]:
train_prob

0     0.083333
8     0.000772
12    0.004018
4     0.000000
2     0.083333
10    0.017761
14    0.022097
6     0.000000
3     0.583333
11    0.961390
15    0.451989
7     0.000000
1     0.250000
9     0.020077
13    0.521896
5     1.000000
Name: prob, dtype: float32

In [24]:
 test_prob

0     0.100000
8     0.001786
12    0.007339
4     0.000000
2     0.000000
10    0.016071
14    0.021101
6     0.000000
3     0.700000
11    0.960714
15    0.461468
7     0.000000
1     0.200000
9     0.021429
13    0.510092
5     1.000000
Name: prob, dtype: float32

# Calculate max steps

In [25]:
def Get_max_steps(P,D):
    import numpy as np

    #generate the transient matrix
    Q = P[:len(D)-1,:len(D)-1]

    #get the determinant of P
    P_det = np.linalg.det(P)
    print("Det:",str(P_det))

    I = np.identity(len(Q))
    M = np.linalg.inv((I-Q))

    M = np.dot(M, np.ones(len(Q)))

    n_steps = np.max(M)

    print("expected steps:",n_steps)

    return n_steps

In [26]:
D = adj_matrix_test.columns.tolist()
D

['Task-Reminder', 'Interaction', 'Email', 'END']

In [27]:
P = adj_matrix_test.values
P

array([[0.1 , 0.  , 0.7 , 0.2 ],
       [0.  , 0.02, 0.96, 0.02],
       [0.01, 0.02, 0.46, 0.51],
       [0.  , 0.  , 0.  , 1.  ]], dtype=float32)

In [28]:
adj_matrix_test

to,Task-Reminder,Interaction,Email,END
from,,,,
Task-Reminder,0.10,0.00,0.70,0.20
Interaction,0.00,0.02,0.96,0.02
Email,0.01,0.02,0.46,0.51
END,0.00,0.00,0.00,1.00


In [29]:
print("train")
Get_max_steps(P=adj_matrix_train.values,D=adj_matrix_train.columns.tolist())
print("\ntest")
Get_max_steps(P=adj_matrix_test.values,D=adj_matrix_test.columns.tolist())

train
Det: -0.0008159999
expected steps: 2.904963357672954

test
Det: -0.00114
expected steps: 2.991551270227804


2.991551270227804

In [30]:
D = adj_matrix_test.columns.tolist()
P = adj_matrix_test.values

#generate the transient matrix
Q = P[:len(D)-1,:len(D)-1]

#get the determinant of P
P_det = np.linalg.det(P)
print("Det:",str(P_det))

I = np.identity(len(Q))
M = np.linalg.inv((I-Q))

M = np.dot(M, np.ones(len(Q)))

n_steps = np.max(M)

print("expected steps:",n_steps)

Det: -0.00114
expected steps: 2.991551270227804


In [31]:
I = np.identity(len(Q))
M = np.linalg.inv((I-Q))

M = np.dot(M, np.ones(len(Q)))

n_steps = np.max(M)

print("expected steps:",n_steps)

expected steps: 2.991551270227804


In [32]:
Get_max_steps(P,D)

Det: -0.00114
expected steps: 2.991551270227804


2.991551270227804

# Bootstrap the whole process

- <b>Problem:</b>
    - Want to figure out whether a _MODEL_ (first-order markov chain), trained on a train split, is equivalent to the empirical distribution in the validation split.

- <b>Hypothesis:</b> 
    - The two adjacency matrices are insignificantly different from each other (approximately the same)

- <b>Experiment:</b> 
    - Run N repetitions of fitting a model from different subsets of the data (bootstrapping)

- <b>Test statistic:</b>
    - T-test, to see wether mew (of all test results) is outside the 95% confidence interval around 0 (where 0 means no difference).

In [33]:
"""
Bootstrapping N runs
"""

kl_errors=[]
euc_errors=[]

for run in range(0, 5000):
    
    ############################################################################
    # ALTERNATIVE MODELS
    ############################################################################
    
    
    
    ############################################################################
    # MARKOV CHAIN
    ############################################################################
    
    #Draw new split from the data
    X_train, X_test = train_test_split(traindata, test_size=0.3, random_state=run)
    
    
    #########################
    # Training
    k_size = 1

    #Train the baseline model
    model_test = pomegranate.MarkovChain.from_samples(X=X_test, k=k_size)
    
    #Train the model
    model_train = pomegranate.MarkovChain.from_samples(X=X_train, k=k_size)
    
    #########################
    # transforming
    test_trans = pd.DataFrame(model_test.distributions[1].to_dict()["table"], columns=["from","to","prob"])
    test_trans["prob"] = pd.to_numeric(test_trans["prob"], downcast="float")
    
    test_trans = test_trans.sort_values(["to", "from"], ascending = (False, False))
    
    test_prob = test_trans["prob"]
    

    train_trans = pd.DataFrame(model_train.distributions[1].to_dict()["table"], columns=["from","to","prob"])
    train_trans["prob"] = pd.to_numeric(train_trans["prob"], downcast="float")
    
    train_trans = train_trans.sort_values(["to", "from"], ascending = (False, False))
    
    train_prob = train_trans["prob"]
    
    # sort them so they have the same order

    
    """
    Check that they are the same?!
    """
    
    #kl_error, euc_error = 999, 999
    #kl_error, euc_error = experiment(X_train, X_test)
    
    if train_trans["from"].values.tolist() == test_trans["from"].values.tolist():
        if train_trans["to"].values.tolist() == test_trans["to"].values.tolist():
            
            #add small value
            train_prob = train_prob + 10e-5
            test_prob = test_prob + 10e-5

            #Normalize
            train_prob = train_prob/np.sum(train_prob)
            test_prob = test_prob/np.sum(test_prob)

            #########################
            # evaluation
            kl_error = kl_divergence(train_prob.values, test_prob.values)
            euc_error = euclidean_distance(train_prob.values, test_prob.values)
    
    
    
            kl_errors.append(kl_error)
            euc_errors.append(euc_error)

KeyboardInterrupt: 

In [ ]:
print(len(kl_errors))

In [ ]:
from scipy import stats

In [ ]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


mew, lower, upper = mean_confidence_interval(kl_errors, 
                                             confidence=0.95)

In [ ]:
import matplotlib.pyplot as plt

a = kl_errors

_ = plt.hist(a, bins='auto')  # arguments are passed to np.histogram

plt.axvline(x = lower, color = 'b', label = 'axvline - full height')
plt.axvline(x = upper, color = 'b', label = 'axvline - full height')

plt.title("Histogram with 'auto' bins")
plt.show()

A p-value larger than a chosen threshold (e.g. 5% or 1%) indicates that our observation is not so unlikely to have occurred by chance. Therefore, we do not reject the null hypothesis of equal population means. If the p-value is smaller than our threshold, then we have evidence against the null hypothesis of equal population means.

In [ ]:
stats.ttest_ind(kl_errors, np.zeros(len(kl_errors)))

P is very very low => H0 must go => they are not similar

In [ ]:
8.775240430092787e-132

## T-test (one sample)

The two hypotheses for this particular one sample t-test are as follows:

H0: µ = 0 (the mean entropy between the two matrices is 0)

HA: µ ≠15 (the mean entropy is not 0)

Because the p-value of our test (0.1201) is greater than alpha = 0.05, we fail to reject the null hypothesis of the test.

We do not have sufficient evidence to say that the mean height for this particular species of plant is different from 15 inches.

In [ ]:
print(stats.ttest_1samp(a=kl_errors, popmean=0))

Because the p-value of our test () is less than alpha = 0.05, we reject the null hypothesis of the test.

We do not have sufficient evidence to say that the mean is 0.

# Alternative models

In [ ]:
#preparing the data for alternative models; here we model 1-step

from_act = []
to_act = []

traces = traindata


for i in range(0, len(traces)):
    
    trace = traces[i]
    
    for activity in range(0,len(trace)):
        
        if activity+1 < len(trace):
        
            t_0 = trace[activity]
            t_1 = trace[activity+1]

            from_act.append(t_0)
            to_act.append(t_1)

In [ ]:
traces[0]

In [ ]:
alt_traindata = pd.DataFrame({"from":from_act,"to":to_act})
alt_traindata

In [ ]:
alt_train_y = alt_traindata["to"]
alt_train_x = alt_traindata["from"] #pd.get_dummies(alt_traindata, columns=["from"]).drop("to",axis=1)

In [ ]:
#Draw new split from the data
x_train, x_test, y_train, y_test = train_test_split(alt_train_x, alt_train_y, test_size=0.3, random_state=run)

x_train_activites = list(set(x_train))

x_train = pd.get_dummies(x_train, columns=["from"]) #.drop("to",axis=1)
x_test = pd.get_dummies(x_test, columns=["from"]) #.drop("to",axis=1)

In [ ]:
#Logistic regression model pipeline
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(multi_class='multinomial', solver='lbfgs')

logisticRegr.fit(x_train, y_train)

predictions = logisticRegr.predict(x_test)

## Generate a transition matrix

In [ ]:
# get unique possible activities from the full dataset
#activities = list(set(np.concatenate(traces).ravel().tolist()))
#activities

In [ ]:
x_train_activites

In [ ]:
"""
Generate a input matrix for every possible (mutually exclusive) combination of input
"""
counterfact = pd.DataFrame({"from":x_train_activites})

counterfact = pd.get_dummies(counterfact, columns=["from"])
counterfact.index = x_train_activites

# sort so that it has ones along diagonal
counterfact = counterfact.sort_values(by=list(counterfact.columns),
    ascending=False,axis=0)

# sorted indexes
counterfact_rows = counterfact.index

#inspect
counterfact

In [ ]:
#generate columns for the transition matrix
columns_new = [w.replace('from', 'to') for w in counterfact.columns]

In [ ]:
transmat = pd.DataFrame(logisticRegr.predict_proba(counterfact),columns=columns_new)
transmat.index = counterfact_rows
transmat

In [ ]:
train_trans = pd.DataFrame(model_train.distributions[1].to_dict()["table"], columns=["from","to","prob"])
train_trans["prob"] = pd.to_numeric(train_trans["prob"], downcast="float")
train_trans

In [ ]:
train_trans["prob"]

# Next steps

1. Wrap procedure in functions so that bootstrapping can be made easier with multiple models
2. Generate test that ensure that matrices are identical (row-order, column-order)
3. Perform the evaluation across multiple models
    - Markov chain (done, baseline)
    - Logistic regression (done)
    - Regularized logistic regression
